# Benchy

Example [benchy](https://build123d.readthedocs.io/en/latest/examples_1.html#examples-benchy)

Downloaded low_poly_benchy.stl using wget:
```sh
wget -O low_poly_benchy.stl https://github.com/gumyr/build123d/blob/dev/examples/low_poly_benchy.stl\?raw\=true
```

In [10]:
from build123d import (
    Axis,
    BuildPart,
    BuildSketch,
    Circle,
    Face,
    Keep,
    Mesher,
    MM,
    Mode,
    Plane,
    Vector,
    Wire,
    add,
    extrude,
    split,
)
from ocp_vscode import show
importer = Mesher()
benchy_stl = importer.read("low_poly_benchy.stl")

show(benchy_stl)

c


In [20]:
with BuildPart() as benchy:
    add(benchy_stl)

    # Determine the plane that defines the top of the roof
    vertices = benchy.vertices()
    roof_vertices = vertices.filter_by_position(Axis.Z, 38, 42)
    roof_plane_vertices = [
        roof_vertices.group_by(Axis.Y, tol_digits=2)[-1].sort_by(Axis.X)[0],
        roof_vertices.sort_by(Axis.Z)[0],
        roof_vertices.group_by(Axis.Y, tol_digits=2)[0].sort_by(Axis.X)[0],
    ]
    roof_plane = Plane(
        Face(Wire.make_polygon([v.to_tuple() for v in roof_plane_vertices]))
    )
    # Remove the faceted smoke stack
    split(bisect_by=roof_plane, keep=Keep.BOTTOM)

    # Determine the position and size of the smoke stack
    smoke_stack_vertices = vertices.group_by(Axis.Z, tol_digits=0)[-1]
    smoke_stack_center = sum(
        [Vector(v.X, v.Y, v.Z) for v in smoke_stack_vertices], Vector()
    ) * (1 / len(smoke_stack_vertices))
    smoke_stack_radius = max(
        [
            (Vector(*v.to_tuple()) - smoke_stack_center).length
            for v in smoke_stack_vertices
        ]
    )

    # Create the new smoke stack
    with BuildSketch(Plane(smoke_stack_center)):
        Circle(smoke_stack_radius)
        Circle(smoke_stack_radius - 2 * MM, mode=Mode.SUBTRACT)
    extrude(amount=-3 * MM)
    with BuildSketch(Plane(smoke_stack_center)):
        Circle(smoke_stack_radius - 0.5 * MM)
        Circle(smoke_stack_radius - 2 * MM, mode=Mode.SUBTRACT)
    extrude(amount=roof_plane_vertices[1].Z - smoke_stack_center.Z)

show(benchy)


/tmp/ipykernel_25105/1619310919.py:13: DeprecationWarning: to_tuple is deprecated and will be removed in a future version. Use 'tuple(Vertex)' instead.
  Face(Wire.make_polygon([v.to_tuple() for v in roof_plane_vertices]))
/tmp/ipykernel_25105/1619310919.py:25: DeprecationWarning: to_tuple is deprecated and will be removed in a future version. Use 'tuple(Vertex)' instead.
  (Vector(*v.to_tuple()) - smoke_stack_center).length


c
